In [49]:
#svm
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [50]:
# Load and preview the data
data = pd.read_csv("labeled_data.csv")
data["labels"] = data["class"].map({0: "Hate Speech", 1: "Offensive Speech", 2: "No Hate and Offensive Speech"})
data = data[["tweet", "labels"]]
print(data.head())

                                               tweet  \
0  !!! RT @mayasolovely: As a woman you shouldn't...   
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   
5  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...   
6  !!!!!!"@__BrighterDays: I can not just sit up ...   

                         labels  
0  No Hate and Offensive Speech  
1              Offensive Speech  
2              Offensive Speech  
3              Offensive Speech  
4              Offensive Speech  
5              Offensive Speech  
6              Offensive Speech  


In [51]:

# Initialize stopwords and stemmer
nltk.download('stopwords')
stopwords_set = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yuvib\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
# Define the clean function
def clean(text):
    text = str(text).lower()
    text = re.sub('[.?]', '', text)
    text = re.sub('https?://\S+|www.\S+', '', text)
    text = re.sub('<.?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopwords_set]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

In [53]:
data["tweet"] = data["tweet"].apply(clean)

x = np.array(data["tweet"])
y = np.array(data["labels"])
cv = CountVectorizer()
X = cv.fit_transform(x)

In [54]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [55]:
# Train and evaluate the SVM model
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.8974202225210905


In [56]:
# Testing the model with example inputs
test_inputs = [
    "You are too bad and I dont like your attitude",
    "It is really awesome",
    "fuck you",
    "you are an idiot",
    "you are killing me",
    "go to hell",
    "you are looking fucking awesome",
    "good morning,","fuck you"
]

for inp in test_inputs:
    inp_transformed = cv.transform([inp]).toarray()
    print(f"Input: {inp}")
    print(f"SVM Prediction: {model.predict(inp_transformed)}")
    print()

Input: You are too bad and I dont like your attitude
SVM Prediction: ['Offensive Speech']

Input: It is really awesome
SVM Prediction: ['No Hate and Offensive Speech']

Input: fuck you
SVM Prediction: ['Offensive Speech']

Input: you are an idiot
SVM Prediction: ['No Hate and Offensive Speech']

Input: you are killing me
SVM Prediction: ['No Hate and Offensive Speech']

Input: go to hell
SVM Prediction: ['Offensive Speech']

Input: you are looking fucking awesome
SVM Prediction: ['No Hate and Offensive Speech']

Input: good morning,
SVM Prediction: ['No Hate and Offensive Speech']

Input: fuck you
SVM Prediction: ['Offensive Speech']



In [57]:

# Save the model
import pickle
pickle.dump(model, open('svm_hate_speech.pkl', 'wb'))

In [58]:
# Save the CountVectorizer
with open('count_vectorizer_svm.pkl', 'wb') as file:
    pickle.dump(cv, file)